In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

# Carregando o dataset (ajuste o caminho do arquivo CSV conforme necessário)
df = pd.read_csv('base_unificada_ate_2015.csv')

# Lista de países únicos no dataset
paises = df['Local'].unique()

# Número de passos de tempo para olhar para trás
n_steps = 5

# Inicializar listas para consolidar dados globais
y_test_global = []
y_pred_global = []

# Função para criar os conjuntos de dados de treinamento e teste
def create_dataset(features, target, n_steps):
    X, y = [], []
    for i in range(len(features) - n_steps):
        X.append(features[i:i + n_steps])
        y.append(target[i + n_steps])
    return np.array(X), np.array(y)

# Loop para treinar o modelo para cada país
for pais in paises:
    # Filtrar os dados para o país
    df_pais = df[df['Local'] == pais]
    
    # Selecionar todas as colunas relevantes (exceto 'Local' e 'Ano') como features
    features = df_pais[['BCG', 'Hib3', 'DTP3', 'Saneamento Seguro (%)', 
                        'Numero Total de Imigrantes', 'Gini (Indice de Desigualdade no Pais)', 
                        'Populacao', 'Taxa de Extrema Pobreza (%)', 'Numero de Usuario de Internet']].values

    # Selecionar a coluna de interesse para prever (número de infecções respiratórias e tuberculose)
    target = df_pais['Numero de Casos de Infeccoes Respiratorios e Tuberculose'].values.reshape(-1, 1)

    # Normalizando os dados (tanto as features quanto o target)
    scaler_features = MinMaxScaler(feature_range=(0, 1))
    scaled_features = scaler_features.fit_transform(features)

    scaler_target = MinMaxScaler(feature_range=(0, 1))
    scaled_target = scaler_target.fit_transform(target)

    # Criando os conjuntos de dados
    X, y = create_dataset(scaled_features, scaled_target, n_steps)

    # Dividindo em conjunto de treinamento e teste (80% para treino, 20% para teste)
    split = int(0.8 * len(X))
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]

    # Reshape para o LSTM [amostras, passos de tempo, características]
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], X_train.shape[2]))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], X_test.shape[2]))

    # Modelo LSTM
    model = Sequential()
    model.add(LSTM(30, activation='relu', input_shape=(n_steps, X_train.shape[2])))
    model.add(Dropout(0.4))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))
    opt = Adam(learning_rate=0.001)
    model.compile(optimizer=opt, loss='mse')

    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-6)

    history = model.fit(X_train, y_train, epochs=200, batch_size=32, 
                        validation_data=(X_test, y_test), callbacks=[early_stopping, reduce_lr])

    # Fazendo previsões
    y_pred = model.predict(X_test)
    y_pred = scaler_target.inverse_transform(y_pred)
    y_test = scaler_target.inverse_transform(y_test.reshape(-1, 1))

    # Consolidando os dados para métricas globais
    y_test_global.extend(y_test.flatten())  # Adiciona valores reais ao array global
    y_pred_global.extend(y_pred.flatten())  # Adiciona previsões ao array global

# Convertendo listas globais para arrays
y_test_global = np.array(y_test_global)
y_pred_global = np.array(y_pred_global)

# Calculando métricas globais
mse_global = mean_squared_error(y_test_global, y_pred_global)
r2_global = r2_score(y_test_global, y_pred_global)

print(f'Mean Squared Error Global: {mse_global}')
print(f'R² Score Global: {r2_global}')


c:\Users\Fernando\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.1043 - val_loss: 0.0103 - learning_rate: 0.0010
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss: 0.1013 - val_loss: 0.0151 - learning_rate: 0.0010
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - loss: 0.0690 - val_loss: 0.0203 - learning_rate: 0.0010
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - loss: 0.0614 - val_loss: 0.0248 - learning_rate: 0.0010
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - loss: 0.0740 - val_loss: 0.0299 - learning_rate: 0.0010
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - loss: 0.0361 - val_loss: 0.0353 - learning_rate: 0.0010
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - loss: 0.0359 - val_loss: 0.0358 - learning_rate: 1.0000e-04
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 0.0705 - val_loss: 0.0364 - learning_rate: 1.0000e-04
Epoch 9/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 0.0508 - val_loss: 0.0370 - learning_rate: 1.0000e-04
Epoch 10/200
1/1 ━

c:\Users\Fernando\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.0760 - val_loss: 0.8519 - learning_rate: 0.0010
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss: 0.1004 - val_loss: 0.8275 - learning_rate: 0.0010
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 0.0725 - val_loss: 0.7991 - learning_rate: 0.0010
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - loss: 0.0701 - val_loss: 0.7680 - learning_rate: 0.0010
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - loss: 0.0743 - val_loss: 0.7340 - learning_rate: 0.0010
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - loss: 0.0624 - val_loss: 0.6995 - learning_rate: 0.0010
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - loss: 0.0603 - val_loss: 0.6668 - learning_rate: 0.0010
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 0.0406 - val_loss: 0.6330 - learning_rate: 0.0010
Epoch 9/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 0.0347 - val_loss: 0.6014 - learning_rate: 0.0010
Epoch 10/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9

c:\Users\Fernando\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.3155 - val_loss: 0.0349 - learning_rate: 0.0010
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 0.3267 - val_loss: 0.0304 - learning_rate: 0.0010
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 0.3029 - val_loss: 0.0267 - learning_rate: 0.0010
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 0.2918 - val_loss: 0.0231 - learning_rate: 0.0010
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 0.3003 - val_loss: 0.0198 - learning_rate: 0.0010
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 0.2607 - val_loss: 0.0167 - learning_rate: 0.0010
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 0.2448 - val_loss: 0.0141 - learning_rate: 0.0010
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 0.2521 - val_loss: 0.0117 - learning_rate: 0.0010
Epoch 9/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 0.2594 - val_loss: 0.0097 - learning_rate: 0.0010
Epoch 10/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6

c:\Users\Fernando\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.6344 - val_loss: 0.6193 - learning_rate: 0.0010
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 0.6578 - val_loss: 0.5859 - learning_rate: 0.0010
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 0.5559 - val_loss: 0.5523 - learning_rate: 0.0010
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 0.5399 - val_loss: 0.5149 - learning_rate: 0.0010
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 0.5091 - val_loss: 0.4765 - learning_rate: 0.0010
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 0.5157 - val_loss: 0.4396 - learning_rate: 0.0010
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 0.4230 - val_loss: 0.4032 - learning_rate: 0.0010
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss: 0.4468 - val_loss: 0.3655 - learning_rate: 0.0010
Epoch 9/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.3684 - val_loss: 0.3301 - learning_rate: 0.0010
Epoch 10/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7

c:\Users\Fernando\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.4730 - val_loss: 0.6007 - learning_rate: 0.0010
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.4735 - val_loss: 0.5370 - learning_rate: 0.0010
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - loss: 0.4431 - val_loss: 0.4745 - learning_rate: 0.0010
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 336ms/step - loss: 0.4358 - val_loss: 0.4136 - learning_rate: 0.0010
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - loss: 0.3119 - val_loss: 0.3521 - learning_rate: 0.0010
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step - loss: 0.3736 - val_loss: 0.2954 - learning_rate: 0.0010
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step - loss: 0.3391 - val_loss: 0.2438 - learning_rate: 0.0010
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - loss: 0.2740 - val_loss: 0.1961 - learning_rate: 0.0010
Epoch 9/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - loss: 0.2023 - val_loss: 0.1534 - learning_rate: 0.0010
Epoch 10/200
1/1 ━━━━━━━━━━━━━━━━━━━━